In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import yaml

In [110]:
cfg = None
with open('config.yaml', 'r') as config:
    cfg = yaml.safe_load(config)["Lab_3"]

In [ ]:
def is_digit(string: str) -> bool:
    if string.isdigit():
       return True
    else:
        try:
            float(string)
            return True
        except ValueError:
            return False

def split_by_spaces(line: str, not_used: list[int], sym_split: str = " ", isDigit: bool = True) -> list[float]:
    line: list[str] = line.split(sym_split)
    line = [s.strip() for s in line.copy() if s != '' and (is_digit(s) if isDigit else True)]
    line = [item for idx, item in enumerate(line) if idx not in not_used]
    return [float(s) if isDigit else s for s in line]

def load_dataset(path: str,
                 columns: list[str],
                 not_used: list[int] = [],
                 sym_split: str = " ",
                 isDigit: bool = True
                 ) -> pd.DataFrame:
    
    with open(path, 'r') as f:
        lines = f.readlines()
    data = [split_by_spaces(line, not_used, sym_split, isDigit) for line in lines]
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [112]:
def analyze_weight(df,
                   alpha = 0.05,
                   critery = stats.kstest,
                   args = [],
                   kwargs = {}
                   ):
    # Точечные оценки среднего и стандартного отклонения
    mean_df = np.mean(df)
    std_df = np.std(df, ddof=1)  # ddof=1 для корректировки стандартного отклонения
    
    results = critery(df, *args, **kwargs)
    stat, p_value = results.statistic, results.pvalue
    n = len(df)
    t_value = stats.t.ppf(1 - alpha / 2, n - 1)
    
    if alpha < p_value:
    # Доверительный интервал для среднего
        mean_ci = (mean_df - t_value * std_df / np.sqrt(n), mean_df + t_value * std_df / np.sqrt(n))
    
    # Доверительный интервал для стандартного отклонения
        std_ci = ((n - 1) * std_df**2 / stats.chi2.ppf(1 - alpha / 2, n - 1),
              (n - 1) * std_df**2 / stats.chi2.ppf(alpha / 2, n - 1))
        
    else:
        mean_ci = None
        std_ci = None
    
    results = {
        "statistic": stat,
        "p_value": p_value,
        "mean_ci": mean_ci,
        "std_ci": std_ci,
    }
    return results

# Dataset "Babyroom"

In [113]:
columns = ["Time of birth recorded on the 24-hour clock",
           "Sex of the child",
           "Birth weight in grams",
           "Number of minutes after midnight of each birth"]

df = load_dataset(cfg['babyroom'], columns, not_used = [])

df["Time of birth recorded on the 24-hour clock"] = df["Time of birth recorded on the 24-hour clock"] / 100

## Гипотеза, что средний вес девочек такой же, как вес мальчиков

In [114]:
girls = df[df['Sex of the child'] == 1]["Birth weight in grams"]
boys = df[df['Sex of the child'] == 2]["Birth weight in grams"]

t, p_value = stats.ttest_ind(girls, boys)
print(f"T-statistic({t:.4f}), P-value({p_value:.4f})")

T-statistic(-1.5229), P-value(0.1353)


$p > 0.05$ => гипотеза верна

## Гипотеза, что дисперсия веса девочек такая же, как и веса мальчиков

In [115]:
print("[Bartlett test]")

stat, p = stats.f_oneway(girls, boys)
print(f"T-statistic({stat:.4f}), P-value({p:.4f})")

[Bartlett test]
T-statistic(2.3191), P-value(0.1353)


$p > 0.05$ => гипотеза верна

# Dataset "iris.txt"

In [116]:
columns = ["Sepal length", "Sepal width", "Petal length", "Petal width", "Class"]

df = load_dataset(cfg['iris'], columns, not_used = [], sym_split=",", isDigit=False)

## Гипотезы о равенстве распределений характеристик цветков разных типов

In [117]:
setosa = df[df['Class'] == 'Iris-setosa']
versicolor = df[df['Class'] == 'Iris-versicolor']
virginica = df[df['Class'] == 'Iris-virginica']

In [118]:
for feature in ['Sepal length', 'Sepal width', 'Petal length', 'Petal width']:
    stat, p_value = stats.kruskal(setosa[feature], versicolor[feature], virginica[feature])
    print(f"{feature:<12}: H-statistic({stat:<8.4f}), P-value({p_value:.4f})")

Sepal length: H-statistic(96.9374 ), P-value(0.0000)
Sepal width : H-statistic(62.4946 ), P-value(0.0000)
Petal length: H-statistic(130.4141), P-value(0.0000)
Petal width : H-statistic(131.0934), P-value(0.0000)


Все $p < 0.05$ => гипотеза не верна

## Гипотезы о равенстве средних и дисперсий различных характеристик цветов разных типов

In [119]:
columns = ['Sepal length', 'Sepal width', 'Petal length', 'Petal width']
def checkMeanAndVar(flower1: pd.DataFrame, flower2: pd.DataFrame) -> dict[list]:
    result = {
        'mean(p-value)': [],
        'var(p-value)': []
    }
    for column in columns:
        t, p_value = stats.ttest_ind(flower1[column].astype(float), flower2[column].astype(float))
        result['mean(p-value)'].append(p_value)
        
        t, p_value = stats.f_oneway(flower1[column].astype(float), flower2[column].astype(float))
        result['var(p-value)'].append(p_value)
    return result

In [120]:
pd.DataFrame(checkMeanAndVar(setosa, versicolor), index=['Sepal length', 'Sepal width', 'Petal length', 'Petal width'])

,mean(p-value),var(p-value)
Sepal length,8.985235e-18,8.985235e-18
Sepal width,4.362239e-15,4.362239e-15
Petal length,5.717464e-62,5.717464e-62
Petal width,4.589081e-56,4.589081e-56


In [121]:
pd.DataFrame(checkMeanAndVar(setosa, virginica), index=['Sepal length', 'Sepal width', 'Petal length', 'Petal width'])

,mean(p-value),var(p-value)
Sepal length,6.892546e-28,6.892546e-28
Sepal width,8.916634e-09,8.916634e-09
Petal length,1.564122e-71,1.564122e-71
Petal width,3.582720e-65,3.582720e-65


In [122]:
pd.DataFrame(checkMeanAndVar(versicolor, virginica), index=['Sepal length', 'Sepal width', 'Petal length', 'Petal width'])

,mean(p-value),var(p-value)
Sepal length,1.724856e-07,1.724856e-07
Sepal width,1.819100e-03,1.819100e-03
Petal length,3.178820e-22,3.178820e-22
Petal width,2.230409e-26,2.230409e-26


Если сравнивать попарно, то ни для одной пары гипотеза не верна

In [ ]:
for feature in ['Sepal length', 'Sepal width', 'Pstats.f_oneway(setosa[feature].astype(float), versicolor[feature].astype(float), virginica[feature].astype(float))etal length', 'Petal width']:
    stat, p = 
    print(f"{feature:<12}: F-statistic({stat:<9.4f}), P-value({p:.4f})")
print()

for feature in ['Sepal length', 'Sepal width', 'Petal length', 'Petal width']:
    stat, p = stats.levene(setosa[feature].astype(float), versicolor[feature].astype(float), virginica[feature].astype(float))
    print(f"{feature:<12}: W-statistic({stat:<9.4f}), P-value({p:.4f})")

Sepal length: F-statistic(119.2645 ), P-value(0.0000)
Sepal width : F-statistic(47.3645  ), P-value(0.0000)
Petal length: F-statistic(1179.0343), P-value(0.0000)
Petal width : F-statistic(959.3244 ), P-value(0.0000)

Sepal length: W-statistic(6.3527   ), P-value(0.0023)
Sepal width : W-statistic(0.6475   ), P-value(0.5248)
Petal length: W-statistic(19.7201  ), P-value(0.0000)
Petal width : W-statistic(19.4122  ), P-value(0.0000)


- Mean:
    - Все: $p < 0.05$
- Var:
    - Sepal width: $p > 0.05$
    - Остальные: $p < 0.05$

# Dataset "sugery.xlsx"

In [124]:
df = pd.read_excel(
    cfg["surgery"],
    names=[
        "V right before operation",
        "V left before operation",
        "V right after operation",
        "V left after operation",
    ],
    skiprows=1,
)

df.head()

,V right before operation,V left before operation,V right after operation,V left after operation
0,7.2,6.70,12.0,13.1
1,1.2,1.20,4.5,4.2
2,6.7,7.30,15.3,14.9
3,9.9,10.05,9.6,9.1
4,3.1,2.13,NaN,NaN


In [125]:
success = (df["V right before operation"] < df["V right after operation"]) & (
    df["V left before operation"] < df["V left after operation"]
)

p7 = stats.binomtest(success.sum(), len(df), p=0.7, alternative="greater").pvalue
p8 = stats.binomtest(success.sum(), len(df), p=0.8, alternative="greater").pvalue
print(f"70% P-value={round(p7, 3)}")
print(f"80% P-value={round(p8, 3)}")

70% P-value=0.275
80% P-value=0.954


$p_{0.7} < 0.05 \\
p_{0.8} > 0.05$

**Вывод:** для 0.8 выполняется, для 0.7 - нет

#  Датасет "euroweight"

In [126]:
columns = ["weight", "batch"]

df = load_dataset(cfg['euroweight'], columns, not_used = [0], sym_split="\t")

In [127]:
df.head(10)

,weight,batch
0,7.512,1.0
1,7.502,1.0
2,7.461,1.0
3,7.562,1.0
4,7.528,1.0
5,7.459,1.0
6,7.518,1.0
7,7.537,1.0
8,7.517,1.0
9,7.605,1.0


In [138]:
columns = ['weight']
def checkMeanAndVar(batch1: pd.DataFrame, batch2: pd.DataFrame) -> float:
    t, p_value = stats.ttest_ind(batch1['weight'].astype(float), batch2['weight'].astype(float))
    return p_value

In [148]:
batches = list(set(df["batch"]))
results = dict.fromkeys(batches)
for batch1 in batches: 
    results[batch1] = dict.fromkeys(batches, None)
    for batch2 in batches:
        results[batch1][batch2] = checkMeanAndVar(df[df['batch'] == batch1], df[df['batch'] == batch2])
        
pd.DataFrame(results)

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
1.0,1.000000,0.261478,1.648508e-03,7.245319e-05,4.999385e-05,1.458601e-01,0.265323,0.356620
2.0,0.261478,1.000000,3.169999e-05,6.710010e-03,5.081426e-03,1.041696e-02,0.960464,0.045903
3.0,0.001649,0.000032,1.000000e+00,2.133669e-12,1.271319e-12,7.165642e-02,0.000021,0.028956
4.0,0.000072,0.006710,2.133669e-12,1.000000e+00,9.119880e-01,2.957552e-08,0.003985,0.000002
5.0,0.000050,0.005081,1.271319e-12,9.119880e-01,1.000000e+00,1.845503e-08,0.002948,0.000001
6.0,0.145860,0.010417,7.165642e-02,2.957552e-08,1.845503e-08,1.000000e+00,0.009122,0.632269
7.0,0.265323,0.960464,2.110993e-05,3.985016e-03,2.947951e-03,9.122232e-03,1.000000,0.043730
8.0,0.356620,0.045903,2.895594e-02,1.604472e-06,1.063086e-06,6.322690e-01,0.043730,1.000000


In [154]:
printed = set()
for batch1 in batches:
    for batch2 in batches:
        if results[batch1][batch2] > 0.05 and batch1 != batch2 and (batch2, batch1) not in printed:
            printed.add((batch1, batch2))
            print(f"batches={(batch1, batch2)} p-value={results[batch1][batch2]}")

batches=(1.0, 2.0) p-value=0.2614778067901794
batches=(1.0, 6.0) p-value=0.14586011949455843
batches=(1.0, 7.0) p-value=0.26532250649811384
batches=(1.0, 8.0) p-value=0.3566197335516895
batches=(2.0, 7.0) p-value=0.9604636345373081
batches=(3.0, 6.0) p-value=0.0716564177786466
batches=(4.0, 5.0) p-value=0.9119879969281968
batches=(6.0, 8.0) p-value=0.6322689805964352


При попарном сравнении у номиналов выше можно считать одинаковое среднее

In [ ]:
groups = [df[df['batch'] == batch]['weight'].values for batch in batches]
stats.f_oneway(*groups)

F_onewayResult(statistic=np.float64(12.67221788627366), pvalue=np.float64(5.361761521220631e-16))

**Вывод:** средние различны